In [1]:
import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
from importlib import reload
# import [module]
# importlib.reload([module])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
def show(x):
    plt.imshow(x)
    plt.show()

import custom.datasets
reload(custom.datasets)
from custom.datasets import nuclei_dataset

In [4]:
from custom.models import u_net
from custom.utils import conditioned_inverse
from custom.datasets import nuclei_dataset, nuclei_dataset_gen
from custom.loss import weighted_bce_loss_with_logits as loss_func

/root/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
nuclei_dataset_df = nuclei_dataset_gen(path='../data/stage1_train/')

In [6]:
dataset = nuclei_dataset(nuclei_dataset_df)

In [7]:
train_loader = DataLoader(dataset,batch_size=8, shuffle=True)
test_loader = DataLoader(nuclei_dataset(nuclei_dataset_df, train=False), batch_size=8)

In [8]:
from custom.models import u_net
model = u_net()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (input, target, weight) in enumerate(train_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [10]:
from torchvision.utils import save_image
def test(epoch):
    model.eval()
    test_loss = 0
    for i, (input, target, weight) in enumerate(test_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        output = model(input)
        test_loss += loss_func(output, target, weight).data[0]
        if i == 0:
            n = 8
            comparison = torch.cat([input[:n], output[:n]])
            save_image(comparison.data.cpu(),
                     './results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [11]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
#


====> Epoch: 1 Average loss: 1.7509
====> Test set loss: 1.7592
====> Epoch: 2 Average loss: 1.7260
====> Test set loss: 1.7690
====> Epoch: 3 Average loss: 1.7304
====> Test set loss: 1.7530
====> Epoch: 4 Average loss: 1.7361
====> Test set loss: 1.7663
====> Epoch: 5 Average loss: 1.7431
====> Test set loss: 1.7574
====> Epoch: 6 Average loss: 1.7414
====> Test set loss: 1.7517
====> Epoch: 7 Average loss: 1.7389
====> Test set loss: 1.7573
====> Epoch: 8 Average loss: 1.7269
====> Test set loss: 1.7472
====> Epoch: 9 Average loss: 1.7416
====> Test set loss: 1.7548
====> Epoch: 10 Average loss: 1.7312
====> Test set loss: 1.7693
